качаем Glove, хороший список моделей тут:

https://github.com/3Top/word2vec-api

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2017-03-07 17:24:12--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.6’

glove.6B.zip.6        0%[                    ] 274.36K  21.8KB/s    eta 8h 22m ^C


In [3]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        fchmod (file attributes) error: Operation not permitted
 (warning) cannot set modif./access times
          Operation not permitted
  inflating: glove.6B.100d.txt       ^C


используем convert.py из  https://raw.githubusercontent.com/jroakes/glove-to-word2vec/master/convert.py
По умолчанию он конвертирует модель размерности 300, мы попробуем размерность 50

In [4]:
!python convert.py

[('zealand', 0.9207794666290283), ('australian', 0.8780614733695984), ('britain', 0.8478989005088806), ('africa', 0.8332697153091431), ('england', 0.8085248470306396), ('united', 0.8037459850311279), ('canada', 0.7994688153266907), ('south', 0.7845566868782043), ('scotland', 0.7658650875091553), ('wales', 0.7653419971466064)]
0.886033768342


In [5]:
import gensim
w2v = gensim.models.KeyedVectors.load_word2vec_format('./glove_model2.txt', binary=False)

используем центроид вместо неизвестных слов

In [16]:
import numpy  as np
centroid = np.mean(w2v.syn0, axis=0)
print(centroid.shape)

(50,)


Качаем архив из http://alt.qcri.org/semeval2015/task2/data/uploads/sts2015-en-post.zip   

Создаем сразу выборки как для RNN, так и для baseline в статье (нужно убедиться что мы все напрогали правильно и RNN работает лучше базовых алгоритмов)

In [6]:
from collections import Counter
def cosine_tokens(tokens1, tokens2):
    c1 = Counter(tokens1)
    c2 = Counter(tokens2)
    intersection = set(c1.keys()) & set(c2.keys())
    numerator = sum([c1[x] * c2[x] for x in intersection])
    sum1 = sum([c1[x]**2 for x in c1.keys()])
    sum2 = sum([c2[x]**2 for x in c2.keys()])
    denominator = np.sqrt(sum1) * np.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    

In [7]:
def jaccard(tokens1, tokens2):
    set1 = set(tokens1)
    set2 = set(tokens2)
    return float(len(set1 | set2))/(len(set1) + len(set2) - len(set1&set2))

In [19]:
from scipy.spatial.distance import cosine
import numpy as np
import re
filename = 'text.clean'
X,Y = [],[]
X_base = []
with open(filename) as inp:    
    for line in inp:    
        sublines = line.split('\t')
        score = int(np.round(float(sublines[0])).astype(np.uint))
        vectors = []
        tokens = []
        for subline in [sublines[4],sublines[5]]:
            subline = re.sub('[^a-z\s]+', ' ', subline.lower())
            tokens.append([t for t in subline.split()])
            vectors.append([w2v[t] if t in w2v.vocab else centroid for t in tokens[-1] ])
        if len(vectors[0])==0 :
            vectors[0].append(centroid)
        if len(vectors[1])==0:
            vectors[1].append(centroid)
        X.append(vectors[0])
        X.append(vectors[1])
        X_base.append([cosine(np.mean(vectors[0], axis=0),np.mean(vectors[1],axis=0)), cosine_tokens(tokens[0], tokens[1]),
                      jaccard(tokens[0], tokens[1])])
        Y.append(score)
idx = np.arange(len(Y))
np.random.shuffle(idx)

X_base = [X_base[i] for i in idx]
Y = [Y[i] for i in idx]
X_new = []
for i in idx:
    X_new.append(X[2*i])
    X_new.append(X[2*i+1])
np.save('semeval_x', X_new)
np.save('semeval_x_baseline', X_base)
np.save('semeval_y', Y)